# Deep Neural Network for MNIST Classification

## Import Relevant Packages

pip install tensorflow_datasets # install if not available in current library and restart kernel

In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds # (import tensorflow data library containing 'mnist dataset')

C:\Users\iolley2\Anaconda3\envs\py3-TF2.0\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [3]:
mnist_dataset,mnist_info = tfds.load(name='mnist',
                         as_supervised=True, # loads data in a 2-tuple structure [inputs,target]
                         with_info=True # provides a tuple containing info about the version, features, # samples of the datasets)
                         )
# a datadet is often downloaded on the computer after the first load (for windows: c-drive/user/username/tensorflow_dataset)

### Extract the test and train dataset

In [4]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']
# No validation dataset

### Extract validation dataset

In [5]:
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples # select 10% of training samples for validation
num_validation_samples = tf.cast(num_validation_samples,tf.int64) #cast/convert the variable into an integer

In [6]:
num_test_samples = 0.1*mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

In [7]:
# Scale data
def scale(image,label):
    image = tf.cast(image,tf.float32)
    image/=255.
    return image,label

In [8]:
scale_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

#### Shuffling data

In [9]:
BUFFER_SIZE=1000
shuffled_train_and_validation_data = scale_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data= shuffled_train_and_validation_data.skip(num_validation_samples)

### Batching

In [10]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)

test_data = test_data.batch(BATCH_SIZE)

validation_inputs,validation_targets=next(iter(validation_data))


## Model

### Outline Model

In [11]:
#784 input, 10 output, 2 hidden layer (50 nodes each)
input_size = 784
output_size = 10
hidden_layer_size = 100

In [17]:
model = tf.keras.Sequential([
    # Input layer
    tf.keras.layers.Flatten(input_shape=(28,28,1)), # flattens/transforms tensor to a vector
    # Hidden Layer
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    # Output Layer
    tf.keras.layers.Dense(output_size,activation='softmax')
])

### Choose optimizer and loss function

In [18]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Training

In [19]:
NUM_EPOCHS = 5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.3305 - accuracy: 0.9064 - val_loss: 0.1816 - val_accuracy: 0.9457 - 10s/epoch - 18ms/step
Epoch 2/5
540/540 - 5s - loss: 0.1359 - accuracy: 0.9604 - val_loss: 0.1274 - val_accuracy: 0.9625 - 5s/epoch - 10ms/step
Epoch 3/5
540/540 - 5s - loss: 0.0955 - accuracy: 0.9715 - val_loss: 0.1075 - val_accuracy: 0.9677 - 5s/epoch - 9ms/step
Epoch 4/5
540/540 - 5s - loss: 0.0719 - accuracy: 0.9783 - val_loss: 0.0958 - val_accuracy: 0.9710 - 5s/epoch - 9ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0568 - accuracy: 0.9829 - val_loss: 0.0977 - val_accuracy: 0.9708 - 5s/epoch - 10ms/step


1. At the beginning of the traininging loss will be set to zero
2. The algorithm will iterate over a pre-set number of batches, all extracted from the training set
3. The weights and biases will be updated as many times as there are batches
4. We'll get a value for the lost function indicating how the training is going
5. We'll also see a training accuracy
6. At the end of the epoch, the algorithm will forward propagate the whole validation data set in a single batch through the optimized model and calculate the validation accuracy
7. When we reach the maximum number of epochs, the training will be over

## Test The Model

In [20]:
test_loss,test_accuracy=model.evaluate(test_data)

100/100 [==============================] - 1s 8ms/step - loss: 0.0833 - accuracy: 0.9742


In [22]:
print('Test Lodd: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100))

Test Lodd: 0.08. Test accuracy: 97.42%
